# 課題（連立方程式，LU分解，QR分解）

- すべての課題を実行せよ
- すべての課題が完了したら Jupyter の File メニューから Download as -> HTML (.html) として結果をダウンロードし， Bb9 の「課題」からファイル添付で提出せよ

In [ ]:
# 初期化なので最初に実行してください
using Test
using LinearAlgebra

## 課題1: 後退代入

- 上三角行列 $U$ と定数ベクトル $b$ に対する下記の連立方程式は $x_n, x_{n-1}, \ldots, x_{1}$ の順番で効率良く解くことができる．これを後退代入（back substitution）を言う．
$$
U x = b
$$


### 課題1-1
- 後退代入によって上三角行列 $U$ と定数ベクトル $b$ からなる連立方程式を解く関数 backsolve を完成させよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように backsolve を修正せよ

#### backsolveの説明
- 引数
    - U: 上三角行列（2次元配列）
    - b: 定数ベクトル
- 戻り値
    - x: $Ux=b$を満たす解
- 関数内の変数
    - U[i,j]: 行列 $U$ の $i,j$ 要素
    - b[i]: ベクトル $b$ の $i$ 要素
    - x[i]: ベクトル $x$ の $i$ 要素
    - n: 未知数の数（ベクトル$b$の長さから取得）

In [ ]:
function backsolve(U, b)
    n = length(b)
    x = zeros(n) # x をゼロで初期化
    for i = n:-1:1 # i = n, n-1, ..., 1 の順番でforループ
        x[i] = b[i] 
        ### Uの非対角要素 U[i,j] に対する処理
        for j = n:-1:i+1 # j = n, n-1, ..., i+2, i+1 の順番でforループ
            ## ここに適切なコードを記述
        end
        ### Uの対角要素 U[i,i] に対する処理
            ## ここに適切なコードを記述
    end
    return x
end

#### backsolveのテストプログラム
- backsolve が作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているのでbacksolveを修正

In [ ]:
for k = 1:10
    s = rand(1:10)
    U = triu(rand(s,s))
    b = rand(s)
    println(@test backsolve(U, b) ≈ U \ b)
end

### 課題1-2
- 関数 backsolve で未知数の数が $n=3$ のときに必要な乗除算と加減算の総数を示せ

#### 課題1-2の解答

- ダブルクリックして Markdown を編集して表に値を記入
- 記入後は `Shift+Enter`

|乗除算|加減算|
|:---:|:---:|
| 0  | 0 | <!-- ここに記入 -->

## 課題2: 前進消去とガウスの消去法

- 一般連立方程式 $Ax=b$ の行列 $A$, ベクトル $b$ からなる拡大係数行列 $[A b]$ に基本変形を施して行列 $A$ を上三角行列にする操作を前進消去（forward elimination）と言う
- 前進消去で作成した上三角行列に関する連立方程式を後退代入で解く手法をガウスの消去法（Gaussian elimination）と言う

### forward!の説明
    - ! は引数の値を変更する関数につける
- 引数
    - A: 行列（2次元配列）
    - b: 定数ベクトル
- 戻り値
    - なし（A, bが直接書き換わる）
- 関数内の変数
    - A[i,j]: 行列 $A$ の $i,j$ 要素
    - b[i]: ベクトル $b$ の $i$ 要素
    - n: 未知数の数（ベクトル$b$の長さから取得）
    - s: 一時的に値を入れる変数

In [ ]:
function forward!(A, b) ## この関数はA, bの値を書き換える
    n = length(b)
    for i = 1:n
        # i 行目を A[i,i] で割る
        s = A[i,i]
        for j = i:n
            A[i,j] /= s
        end
        b[i] /= s
        # k = i+1 から n 行目の i 番目要素を0にする
        for k = i+1:n
            s = A[k,i]
            for j = 1:i
                A[k,j] = 0.0
            end
            for j = i+1:n
                A[k,j] -= A[i,j] * s
            end
            b[k] -= b[i] * s
        end
    end
end

### 課題2-1
- 関数 forward! で未知数の数が $n=3$ のときに必要な乗除算と加減算の総数を示せ

#### 課題2-1の解答

- ダブルクリックして Markdown を編集して表に値を記入
- 記入後は `Shift+Enter`

|乗除算|加減算|
|:---:|:---:|
| 0 | 0 |

### 課題2-2
- forward!を実行してみる

In [ ]:
## 乱数で5x5の行列Aとベクトルbを作成
A = rand(5,5)
b = 　rand(5)
display(A)
display(b)

## forward!の実行
## 副作用（引数の値が変化する）があるのでtmpA, tmpbにコピーして実行
tmpA = copy(A)
tmpb = copy(b)
forward!(tmpA, tmpb)
display(tmpA)
display(tmpb)

### gesolveの説明
- 引数
    - A: 行列（2次元配列）
    - b: 定数ベクトル
- 戻り値
    - x: Ax=bの解
- 関数内の変数
    - tmpA: 行列 $A$ のコピー
    - tmpb: ベクトル $b$ のコピー

In [ ]:
function gesolve(A, b)
    tmpA = copy(A)
    tmpb = copy(b)
    forward!(tmpA, tmpb)
    backsolve(tmpA, tmpb) # 最後に評価したものが戻り値になる
end

### 課題2-3
- ガウスの消去法で連立方程式を解く関数 gesolve と組込のソルバーを比較
- それぞれで得た解 x1, x2 の誤差 r1, r2 を算出
    - 違いがあるかないか，違いがあるならなぜ違いが生じたのか考察
- 連立方程式の誤差の定義： $Ax-b$ を計算して絶対値が 0 に近いかどうか
$$
r = max|A x - b|
$$

In [ ]:
## A, b の表示
n = 100
A = rand(n,n)
b = rand(n)

## gesolve による解 x1, Julia 標準による解 x2
x1 = gesolve(A, b)
x2 = A \ b

## 誤差を計算
r1 = maximum(abs.(A*x1-b))
r2 = maximum(abs.(A*x2-b))
display(r1)
display(r2)

#### 課題2-3の解答
- 考察はこのセルに記述（ダブルクリックしてMarkdownで記述してから Shift+Enter）

## 課題3：前進消去その２
- A が正方行列でない場合は不定（解が一つに定まらない）や不能（解がない）になることが多い
- forward!を実行して不能の場合に前進消去した結果がどのようになるか検証せよ

### forward!の説明
- 引数
    - A: 行列（2次元配列）
    - b: 定数ベクトル
- 戻り値
    - なし（A, bが直接書き換わる）
- 関数内の変数
    - A[i,j]: 行列 $A$ の $i,j$ 要素
    - b[i]: ベクトル $b$ の $i$ 要素
    - m: 条件式の数（行列$A$の行数から取得）
    - n: 未知数の数（行列$A$の列数から取得）
    - s: 一時的に値を入れる変数

In [ ]:
## forward! でAが (m, n) 行列の場合に対応させる
function forward!(A, b)
    m,n = size(A)
    for i = 1:min(m,n)
        s = A[i,i]
        for j = i:n
            A[i,j] /= s
        end
        b[i] /= s
        for k = i+1:m
            s = A[k,i]
            for j = 1:i
                A[k,j] = 0.0
            end
            for j = i+1:n
                A[k,j] -= A[i,j] * s
            end
            b[k] -= b[i] * s
        end
    end
end

### 課題3-1
- 前のforward!と比べて変更があったところをすべて挙げよ

#### 課題3-1の解答
- n=length(b) から m,n = size(A) に変わった
- etc.

### 課題3-2
- 不能（解なし）となる連立方程式を作成せよ

In [ ]:
## 行列 A, ベクトル b の定義: 値は1.0のように小数表記
A = [
    ## 適当な行列を作成
]
b = [] ## 適当なベクトルを作成

## rank関数を使って解があるかどうか判定
if rank(A) == rank([A b])
    println("解あり")
else
    println("解なし")
end

### 課題3-3
- 課題3-2で定義したA, b に対してforward!を実行せよ
- forward!の実行結果からrank関数を使わずに「不能」を判定するためにはどうしたらよいか処理の方針を記述せよ

In [ ]:
## forward!の実行
tmpA = copy(A)
tmpb = copy(b)
forward!(tmpA, tmpb)
display(tmpA)
display(tmpb)

#### 課題3-3の解答
- 考察はこのセルに記述（ダブルクリックしてMarkdownで記述してから `Shift+Enter`）

## 課題4: 直交化とQR分解
- 線型空間上にあるベクトルの組から互いに直交するベクトルの組を生成することを直交化（Orthogonalization）と言う．
- グラム・シュミットの直交化法
    - 与えられたベクトルの組 $v_1, v_2, \ldots, v_n$ から互いに直交し，かつ長さ1のベクトルの組 $u_1, u_2, \ldots, u_n$ を生成する
- 行列 $A$ を直交行列 $Q$ と上三角行列 $R$ に分解する手法を OR 分解（QR decomposition）と言う
$$
A = QR
$$

### cgs (Classical Gram–Schmidt orthonormalization) の説明
- 引数
    - A: m行n列の行列,　n個のベクトルの組
- 戻り値
    - Q: m行n列の行列，n個の正規直交ベクトルの組
- 関数内の変数
    - A[:,j]: 行列 $A$ の $j$ 列目のベクトル
    - Q[:,j]: 行列 $Q$ の $j$ 列目のベクトル
    - m: ベクトルの次元
    - n: ベクトルの本数
    - w: 内積を一時的に格納する変数

In [ ]:
function cgs(A)
    m,n = size(A)
    Q = copy(A)  # Aのコピーを作成
    Q[:,1] /= norm(Q[:,1]) # Q[:,1]のノルムで全体を割る
    for k = 2:n
        for i = 1:k-1
            w = dot(Q[:,i], A[:,k]) # すでにできている正規直交基底 Q[:,1], ..., Q[:k-1] と A[:,k] の内積を計算
            Q[:,k] -= w * Q[:,i]
        end
        Q[:,k] /= norm(Q[:,k])
    end
    return Q
end

### 課題4-1
- 関数 cgs で行列 $A$ が 3 行 3 列のときに必要な乗除算と加減算の総数を示せ．ただし，normそのものの計算に必要な乗除算と加減算の回数は除く

#### 課題4-1の解答

- ダブルクリックして Markdown を編集して表に値を記入
- 記入後は `Shift+Enter`

|乗除算|加減算|
|:---:|:---:|
| 0 | 0 |

### 課題4-2
- 関数 cgs を実行して直交行列Qを得よ
- Q' * Q を計算し，その結果を誤差の観点から考察せよ

In [ ]:
A = rand(5,5) ## 適当な行列を作成
Q = cgs(A) ## 直交化した行列を作成
B = Q' * Q ## 転置した行列との積
display(B)

#### 課題4-2の解答

- 考察はこのセルに記述（ダブルクリックしてMarkdownで記述してから Shift+Enter）

### qr2 の説明
- 引数
    - A: m行n列の行列
- 戻り値
    - Q: 互いに直交するn個の列ベクトルからなる行列．m行n列の行列
    - R: n行n列の上三角行列
- 関数内の変数
    - A[:,j]: 行列 $A$ の $j$ 列目のベクトル
    - Q[:,j]: 行列 $Q$ の $j$ 列目のベクトル
    - R[i,j]: 行列 $R$ の $i,j$ 要素
    - m: 行数
    - n: 列数

In [ ]:
function qr2(A)
    m,n = size(A)
    Q = copy(A)  # Aのコピーを作成
    R = zeros(n,n)
    R[1,1] = norm(Q[:,1])
    Q[:,1] /= R[1,1]
    for k = 2:n
        for i = 1:k-1
            R[i,k] = dot(Q[:,i], A[:,k])
            Q[:,k] -= R[i,k] * Q[:,i]
        end
        R[k,k] = norm(Q[:,k])
        Q[:,k] /= R[k,k]
    end
    return (Q,R)
end

### 課題4-3
- cgsとqr2を比較して変更があったところをすべて挙げよ

#### 課題4-3の解答
- R = zeros(n,n)を追加
- etc.

### 課題4-4
- qr2 を用いて QR 分解を実行せよ

In [ ]:
## QR分解をやってみる
A = rand(5,5)
Q, R = qr2(A)
display(Q)
display(R)

### 課題4-5
- backsolveとqr2を利用してQR分解で連立方程式を解く関数 qrsolve を作成せよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように関数を修正せよ

#### qrsolveの説明
- 引数
    - A: 正方行列（2次元配列）
    - b: 定数ベクトル
- 戻り値
    - x: Ax=bの解

In [ ]:
## qrsolveの定義
function qrsolve(A, b)
    ## ここにコードを記述
end

#### 課題4-4のテストプログラム
- qrsolve が作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認してください
- **Test Failed** が出たら結果が計算が間違っているので修正してください

In [ ]:
for k = 1:10
    s = rand(1:10)
    A = rand(s,s)
    b = rand(s)
    println(@test qrsolve(A, b) ≈ A \ b)
end